<a href="https://colab.research.google.com/github/karim306/Flowers-Classfier/blob/main/FlowerClassfier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "kareemahm3d" # username from the json file
os.environ['KAGGLE_KEY'] = "xxxx" # key from the json fil

# Download The DataSet


In [2]:
!kaggle datasets download -d kausthubkannan/5-flower-types-classification-dataset

Dataset URL: https://www.kaggle.com/datasets/kausthubkannan/5-flower-types-classification-dataset
License(s): other
 91% 221M/242M [00:02<00:00, 78.0MB/s]
100% 242M/242M [00:02<00:00, 100MB/s] 


# Unzip the dataset


In [3]:
# Unzip the dataset into the same directory
!unzip -q 5-flower-types-classification-dataset.zip -d ./5-flower-dataset


# **Split The Data**

In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Original dataset directory
base_dir = './5-flower-dataset'  # Replace with the actual dataset directory

# Paths for the new directories outside of the original dataset folder
output_base = './split_data'  # Directory to contain train, val, and test directories
train_dir = os.path.join(output_base, 'train')
val_dir = os.path.join(output_base, 'val')
test_dir = os.path.join(output_base, 'test')

# Create the directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get the classes (subfolders) from the original dataset
classes = [folder for folder in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, folder))]
for cls in classes:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)


In [9]:
# Split data into train, val, and test sets (70% train, 20% val, 10% test)
for cls in classes:
    cls_path = os.path.join(base_dir, cls)
    images = [f for f in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, f))]

    if len(images) == 0:
        print(f"Warning: No images found for class '{cls}'. Skipping.")
        continue  # Skip empty directories

    # Split into train and remaining (val + test)
    train_images, val_test_images = train_test_split(images, test_size=0.3, random_state=42)
    # Split val + test into separate validation and test sets
    val_images, test_images = train_test_split(val_test_images, test_size=0.33, random_state=42)  # 33% of 30% = 10%

    # Move images to respective directories
    for img in train_images:
        shutil.copy(os.path.join(cls_path, img), os.path.join(train_dir, cls, img))
    for img in val_images:
        shutil.copy(os.path.join(cls_path, img), os.path.join(val_dir, cls, img))
    for img in test_images:
        shutil.copy(os.path.join(cls_path, img), os.path.join(test_dir, cls, img))


In [13]:
import shutil
import os

# Path to the split directories
for dir_path in [train_dir, val_dir, test_dir]:
    for root, dirs, files in os.walk(dir_path):
        for dir_name in dirs:
            # Check if the folder name is 'train', 'val', or 'test'
            if dir_name in ['train', 'val', 'test','flower_images']:
                unwanted_dir = os.path.join(root, dir_name)
                shutil.rmtree(unwanted_dir)  # Remove the unwanted directory
                print(f"Removed unwanted folder: {unwanted_dir}")


Removed unwanted folder: ./split_data/train/flower_images
Removed unwanted folder: ./split_data/val/flower_images
Removed unwanted folder: ./split_data/test/flower_images


# **Class Names**

In [14]:
import os

def class_name_from_folder(folder_path):
  """
  Extracts class names from subfolders within a given folder.

  Args:
    folder_path: The path to the folder containing class subfolders.

  Returns:
    A list of class names (subfolder names).
  """
  return [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

class_name = class_name_from_folder('/content/split_data/train')
class_name

['Tulip', 'Sunflower', 'Lotus', 'Orchid', 'Lilly']